In [5]:
import tweepy
import configparser
from nltk.stem import SnowballStemmer
import pandas as pd
import json
import requests
import numpy as np
import csv
import re 
import string
import preprocessor as p
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score 
import pickle
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer  
import spacy
from sklearn import *
nlp=spacy.load('es_core_news_lg')
spanish_stemmer = SnowballStemmer('spanish')

CONSUMER_KEY='sj7r6f0zF37UB35yiFUWhG3mw'
CONSUMER_SECRET='kciblvx1oHsKJn0ggPxuswSJBeMgeU1vSFiqTmqGgytwHX2G4H'
ACCESS_KEY='1439327344536264711-GegKaB1npGPB4vubPVPzZHjaQywGBl'
ACCESS_SECRET='RsSRxRUOClFgh9RqWX3kxq3HV9FiyDVE4YTyGdvYwvN1I'
# authentication
"""auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)"""
import emoji

In [6]:
BEARER='AAAAAAAAAAAAAAAAAAAAADtbdgEAAAAA3jZdkB0mA%2BzRvmohygx%2BQAYgF6g%3DIRBwQuUHUatX216RMyG7AdjxLZWHTGIjcaoJ3nzhl0HDwYWmTB'     

## Descarga de nuevos Tweets para hacer la clasificacion de sentimientos

In [7]:
client = tweepy.Client(bearer_token=BEARER, consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET,
                       access_token=ACCESS_KEY, access_token_secret=ACCESS_SECRET,return_type = requests.Response,  wait_on_rate_limit=True)    

In [8]:
def mandar_tweet():
    for i in range(2):
        query = "Elon Musk  lang:es -is:retweet"


        # your start and end time for fetching tweets


        # get tweets from the API
        tweets = client.search_recent_tweets(query=query,
                                             tweet_fields = ["created_at", "text", "source"],
                                             user_fields = ["name", "username", "description"],
                                             max_results = 100,
                                             expansions='author_id'
                                             )


        tweets_dict = tweets.json() 

        # Extract "data" value from dictionary
        tweets_data = tweets_dict['data'] 

        # Transform to pandas Dataframe
        df = pd.json_normalize(tweets_data) 

        normalized_places = pd.json_normalize(tweets_dict['includes']['users']).set_index('id')
        df2=pd.merge(normalized_places,df,left_on='id',right_on='author_id')

        df2=df2.drop(["description","author_id","id","source","name"],axis=1)  

        df2["text"][5]
        df2

        df2["created_at"]="20220616"


            #saco links
        df2["text"] = df2["text"].replace({r'(http|https|\n\nhttps|\nhttps):\/\/([^\/\r\n]+)(\/[^\r\n]*)?':' '}, regex=True)
        df2["text"] = df2["text"].replace({r'\W+':' '}, regex=True)


        df3=pd.DataFrame()
        df3.insert(0, 'fecha', df2.created_at)
        df3.insert(1, 'texto', df2.text)
        df3.insert(2, 'usuario', df2.username)


        # Converting first column to Series
        fecha = df3.iloc[:,0]
        texto = df3.iloc[:,1]
        usuario = df3.iloc[:,2]
        url = 'https://rutbmzlxuu8vw3w-db20220601185600.adb.sa-saopaulo-1.oraclecloudapps.com/ords/pln/pln/registrar'

    for i in range(len(fecha)):
        texto[i]=texto[i].strip()
        x = requests.post(url,headers = {"fecha": str(fecha[i]),"texto":str(texto[i]),"usuario":str(usuario[i])})
        print(x.text)
    print("__________________________________________________________")

In [9]:
def test_clasificador():
    global comparar
    global df3
    for i in range(1):
        query = "Elon Musk  lang:es -is:retweet"


        # your start and end time for fetching tweets


        # get tweets from the API
        tweets = client.search_recent_tweets(query=query,
                                             tweet_fields = ["created_at", "text", "source"],
                                             user_fields = ["name", "username", "description"],
                                             max_results = 10,
                                             expansions='author_id'
                                             )


        tweets_dict = tweets.json() 

        # Extract "data" value from dictionary
        tweets_data = tweets_dict['data'] 

        # Transform to pandas Dataframe
        df = pd.json_normalize(tweets_data) 

        normalized_places = pd.json_normalize(tweets_dict['includes']['users']).set_index('id')
        df2=pd.merge(normalized_places,df,left_on='id',right_on='author_id')

        df2=df2.drop(["description","author_id","id","source","name"],axis=1)  


            #saco links
        df2["text"] = df2["text"].replace({r'(http|https|\n\nhttps|\nhttps):\/\/([^\/\r\n]+)(\/[^\r\n]*)?':' '}, regex=True)
        df2["text"] = df2["text"].replace({r'\W+':' '}, regex=True)
        df2["text"] = df2["text"].replace({r'[,.:?’_¡—“¿%;!/`''©@)°C(\"\n]':' '}, regex=True)
        df2["text"] = df2["text"].replace({r'[0-9]':' '}, regex=True)
        

        df3=pd.DataFrame()
 
        df3.insert(0, 'Tweet', df2.text)
        comparar=df3.copy()
    return(df3)

## Pipeline de pre procesamiento de texto

In [11]:
 def stopWords(corpus):
    global oraciones
    global oraciones_filtradas
    f = open('StopWords.txt', 'r')
    stopwords = f.read().split('\n')
    f.close()
    " ".join(stopwords)
    oraciones=list(corpus)

    oraciones_filtradas = [" ".join([
                              palabra for palabra in oracion.split()
                              if palabra not in stopwords])
                              for oracion in oraciones]
stopWords(df3["Tweet"])

In [12]:
spanish_stemmer = SnowballStemmer('spanish')
oraciones_filtradas_lematizadas=[]
for oracion in oraciones_filtradas:
    oracion_lematizada = []
    for palabra in oracion.split():
        doc=nlp(palabra)
        oracion_lematizada.append(doc[0].lemma_)
    oraciones_filtradas_lematizadas.append(" ".join(oracion_lematizada))
    

oraciones_filtradas_lematizadas_stemm=[]

for oracion in oraciones_filtradas_lematizadas:
    oracion_stemm = []
    for palabra in oracion.split():
        oracion_stemm.append(spanish_stemmer.stem(palabra))
    oraciones_filtradas_lematizadas_stemm.append(" ".join(oracion_stemm))
    
df3['Tweet'] = oraciones_filtradas_lematizadas_stemm
df3 = df3.drop_duplicates().reset_index(drop=True)
 

In [13]:
df3=df3.replace('', np.nan).dropna()
 

In [23]:
# Dataset listo
df3

,Tweet
0,the spy who lov yo bond moor mas floj uno vill...
1,jesusrener record ciert cre tesl fundador este...
2,algun vez hab vist escuch biograf monstru nego...
3,omo elon musk hab hund todav mas twitt poz perd
4,sinline mx seis millon bot exceptu asquer fami...
5,sinline mx por compr elon musk twitt gent com ...
6,lueg vari mes incertidumbr sobr compr twitt pa...
7,a men ver bail asi hij elon musk
8,gulfstream g asi dentr nuev avion elon musk lu...


## Clasificacion 

In [15]:
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.feature_extraction.text import CountVectorizer
global result
global probabilidad
corpus=df3["Tweet"]
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("cuerpo.pkl", "rb"))) 
new_input=loaded_vec.transform(corpus)
new_input=pd.DataFrame(new_input.toarray()) 


clf_input=pickle.load(open("prediccion.sav", "rb"))
predict_input=clf_input.predict(new_input)

In [16]:
predict_input=pd.DataFrame(predict_input)
predict_input.columns=["clase"]

In [17]:
predict_input['clase']= predict_input['clase'].replace(1,"Positivo")
predict_input['clase']= predict_input['clase'].replace(0,"Negativo")

In [18]:
 predict_input

,clase
0,Negativo
1,Negativo
2,Positivo
3,Negativo
4,Negativo
5,Positivo
6,Positivo
7,Positivo
8,Negativo


In [19]:
 predict_input.insert(1, 'Tweet', comparar.Tweet)

## RESULTADOS

In [20]:
 
display(predict_input)

,clase,Tweet
0,Negativo,The Spy Who Loved Me es el Bond de Moore más ...
1,Negativo,jesusrenero Recordemos por cierto que no creó...
2,Positivo,Alguna vez has visto o escuchado la biografía...
3,Negativo,ómo Elon Musk ha hundido todavía más a Twitte...
4,Negativo,SinLinea Mx Seis millones de bots Exceptuando...
5,Positivo,SinLinea Mx Por eso no lo compro Elon Musk a ...
6,Positivo,Luego de varios meses de incertidumbre sobre l...
7,Positivo,A menos que la vea bailando asi con un hijo de...
8,Negativo,Gulfstream G así es por dentro el nuevo avi...


In [21]:
predict_input.to_csv("prediccion.csv")